In [152]:
import pandas as pd
import numpy as np


In [186]:
data_df = pd.read_csv(r'/Users/zoeyneo/code/zoeyneo/investoinsight/cleaned_time_series.csv')
data_df

,Unnamed: 0,Date,Symbol,Dividend,Volume,fed_funds_rate,GDP,debt_to_equity,EPS,return_on_equity,quick ratio,operating_ratio,inventory_turnover,Stock Price
0,29538,2018-07-31,HPQ,0.00,6.845513e+06,1.91,4689.905000,-20.385399,0.556258,-0.498019,0.614953,0.925134,1.953374,19.810163
1,29037,2018-08-31,CCL,0.50,3.182053e+06,1.91,4717.323667,0.693559,2.417847,0.069238,0.168329,0.692598,7.303704,55.871806
2,29042,2018-08-31,FDS,0.64,2.294333e+05,1.91,4717.323667,1.699082,1.801999,0.130867,1.920293,1.234522,27.010998,203.856165
3,29043,2018-08-31,FDX,0.00,1.211117e+06,1.91,4717.323667,1.706984,3.150943,0.043551,1.261503,0.378313,7.585086,224.475233
4,29045,2018-08-31,HPQ,0.00,6.493083e+06,1.91,4717.323667,-25.713635,0.679721,-0.769470,0.613254,0.927927,1.998739,20.428785
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13028,25460,2023-03-31,XYL,0.00,1.532100e+06,4.65,5053.623333,1.216042,0.548780,0.027669,1.441098,0.931630,1.052509,100.078591
13029,25461,2023-03-31,YUM,0.00,1.474093e+06,4.65,5053.623333,-1.655231,1.067616,-0.034192,0.709968,0.435866,0.911765,126.797676
13030,25462,2023-03-31,ZBH,0.24,1.147900e+06,4.65,5053.623333,0.747768,1.110315,0.019135,0.929235,0.820884,0.223442,124.411347
13031,25464,2023-03-31,ZION,0.00,5.970143e+06,4.65,5053.623333,16.085841,1.377448,0.039352,0.296107,0.615894,-17.000000,38.069854


In [187]:
data_df['Date'] = pd.to_datetime(data_df['Date'])

# Extract both month and year from the 'date' column
data_df['month'] = data_df['Date'].dt.month
data_df['year'] = data_df['Date'].dt.year

# Combine 'month' and 'year' into 'month_year'
data_df['Date'] = data_df.apply(lambda x: f"{x['year']}-{x['month']:02d}", axis=1)

data_df.drop(columns=['month', 'year'], inplace=True)
data_df.head()

,Unnamed: 0,Date,Symbol,Dividend,Volume,fed_funds_rate,GDP,debt_to_equity,EPS,return_on_equity,quick ratio,operating_ratio,inventory_turnover,Stock Price
0,29538,2018-07,HPQ,0.00,6.845513e+06,1.91,4689.905000,-20.385399,0.556258,-0.498019,0.614953,0.925134,1.953374,19.810163
1,29037,2018-08,CCL,0.50,3.182053e+06,1.91,4717.323667,0.693559,2.417847,0.069238,0.168329,0.692598,7.303704,55.871806
2,29042,2018-08,FDS,0.64,2.294333e+05,1.91,4717.323667,1.699082,1.801999,0.130867,1.920293,1.234522,27.010998,203.856165
3,29043,2018-08,FDX,0.00,1.211117e+06,1.91,4717.323667,1.706984,3.150943,0.043551,1.261503,0.378313,7.585086,224.475233
4,29045,2018-08,HPQ,0.00,6.493083e+06,1.91,4717.323667,-25.713635,0.679721,-0.769470,0.613254,0.927927,1.998739,20.428785


In [189]:
data_df.rename(columns={'Date': 'date','Symbol':'symbol', 'Volume':'volume','GDP':'gdp', 'EPS':'eps','quick ratio':'quick_ratio','Stock Price':'stock_price'}, inplace=True)

In [190]:
column_names = data_df.columns
print(column_names)

Index(['Unnamed: 0', 'date', 'symbol', 'Dividend', 'volume', 'fed_funds_rate',
       'gdp', 'debt_to_equity', 'eps', 'return_on_equity', 'quick_ratio',
       'operating_ratio', 'inventory_turnover', 'stock_price'],
      dtype='object')


Merge sentiment data with time series data

In [123]:
sa18_df = pd.read_csv(r'/Users/zoeyneo/code/zoeyneo/investoinsight/sentiment2018.csv')
sa18_df.head()

,Unnamed: 0,Headline,Date,Stock,Neutral,Positive,Negative
0,0,50 biggest movers from yesterday,2018-11-21,A,8.389207e-01,3.987503e-02,1.212042e-01
1,1,36 stocks moving in tuesdays midday session,2018-11-20,A,9.999959e-01,1.325494e-07,3.961927e-06
2,2,shares of agilent technologies are trading up ...,2018-11-20,A,4.851770e-08,1.000000e+00,1.385383e-09
3,3,morgan stanley maintains overweight on agilent...,2018-11-20,A,3.539342e-07,9.999996e-01,2.367040e-08
4,4,26 stocks moving in tuesdays premarket session,2018-11-20,A,9.999979e-01,1.054393e-07,2.009748e-06


In [124]:
df = pd.DataFrame(sa18_df)

# Convert 'date' column 
df['date'] = pd.to_datetime(df['Date'], format='%Y-%m')

# Extract both month and year from the 'date' column
df['month'] = df['date'].dt.month
df['year'] = df['date'].dt.year

# Combine 'month' and 'year' into 'month_year'
df['month_year'] = df.apply(lambda x: f"{x['year']}-{x['month']:02d}", axis=1)

# Calculate the average score by month
average_by_month_year = df.groupby(['Stock','month_year'])[['Neutral', 'Positive', 'Negative']].mean()

# Save the result to a new DataFrame
SA_df = average_by_month_year.reset_index()

# Print the result DataFrame
print(SA_df)

     Stock month_year   Neutral  Positive  Negative
0        A    2018-01  0.833321  0.166669  0.000010
1        A    2018-02  0.666658  0.333336  0.000007
2        A    2018-03  0.999707  0.000251  0.000041
3        A    2018-04  0.770916  0.193607  0.035477
4        A    2018-05  0.805719  0.115470  0.078811
...    ...        ...       ...       ...       ...
3951   ZTS    2018-07  0.881442  0.118524  0.000034
3952   ZTS    2018-08  0.685380  0.258737  0.055883
3953   ZTS    2018-10  0.500005  0.499992  0.000003
3954   ZTS    2018-11  0.749961  0.249970  0.000070
3955   ZTS    2018-12  0.986359  0.013577  0.000065

[3956 rows x 5 columns]


In [125]:
sa19a_df = pd.read_csv(r'/Users/zoeyneo/code/zoeyneo/investoinsight/sentiment2019p1.csv')
sa19a_df.head()

,Unnamed: 0,Headline,Date,Stock,Neutral,Positive,Negative
0,0,shares of many healthcare companies are tradin...,2019-05-16,A,0.008474,8.995113e-01,0.092015
1,1,60 biggest movers from yesterday,2019-05-16,A,0.954836,4.759314e-03,0.040405
2,2,44 stocks moving in wednesdays midday session,2019-05-15,A,0.999997,1.212762e-07,0.000003
3,3,benzinga pros top 10 mostsearched tickers for ...,2019-05-15,A,0.999995,1.778685e-06,0.000003
4,4,bank of america maintains buy on agilent techn...,2019-05-15,A,0.000008,9.998995e-01,0.000093


In [126]:
df = pd.DataFrame(sa19a_df)

# Convert 'date' column 
df['date'] = pd.to_datetime(df['Date'], format='%Y-%m')

# Extract both month and year from the 'date' column
df['month'] = df['date'].dt.month
df['year'] = df['date'].dt.year

# Combine 'month' and 'year' into 'month_year'
df['month_year'] = df.apply(lambda x: f"{x['year']}-{x['month']:02d}", axis=1)

# Calculate the average score by month
average_by_month_year = df.groupby(['Stock','month_year'])[['Neutral', 'Positive', 'Negative']].mean()

# Save the result to a new DataFrame
test_df = average_by_month_year.reset_index()

# Print the result DataFrame
# print(test_df)

# Concatenate vertically (add df2 to the bottom of df1)
SA_df = pd.concat([SA_df, test_df], axis=0)

# Reset the index
SA_df.reset_index(drop=True, inplace=True)

# Print the concatenated DataFrame
print(SA_df)

     Stock month_year   Neutral  Positive  Negative
0        A    2018-01  0.833321  0.166669  0.000010
1        A    2018-02  0.666658  0.333336  0.000007
2        A    2018-03  0.999707  0.000251  0.000041
3        A    2018-04  0.770916  0.193607  0.035477
4        A    2018-05  0.805719  0.115470  0.078811
...    ...        ...       ...       ...       ...
5587   ZTS    2019-01  0.951553  0.002659  0.045788
5588   ZTS    2019-02  0.775619  0.224061  0.000320
5589   ZTS    2019-03  0.866393  0.125151  0.008456
5590   ZTS    2019-04  0.615322  0.383756  0.000922
5591   ZTS    2019-05  0.795156  0.200077  0.004767

[5592 rows x 5 columns]


In [127]:
sa19b_df = pd.read_csv(r'/Users/zoeyneo/code/zoeyneo/investoinsight/sentiment2019p2.csv')
sa19b_df.head()

,Unnamed: 0,Headline,Date,Stock,Neutral,Positive,Negative
0,0,stocks that hit 52week highs on friday,2019-12-27,A,0.996414,0.000019,0.003567
1,1,stocks that hit 52week highs on monday,2019-12-23,A,0.997260,0.000010,0.002729
2,2,stocks that hit 52week highs on friday,2019-12-20,A,0.996414,0.000019,0.003567
3,3,stocks that hit 52week highs on thursday,2019-12-12,A,0.998194,0.000012,0.001794
4,4,theres a new trading tool that allows traders ...,2019-12-11,A,0.998351,0.001628,0.000021


In [128]:
df = pd.DataFrame(sa19b_df)

# Convert 'date' column 
df['date'] = pd.to_datetime(df['Date'], format='%Y-%m')

# Extract both month and year from the 'date' column
df['month'] = df['date'].dt.month
df['year'] = df['date'].dt.year

# Combine 'month' and 'year' into 'month_year'
df['month_year'] = df.apply(lambda x: f"{x['year']}-{x['month']:02d}", axis=1)

# Calculate the average score by month
average_by_month_year = df.groupby(['Stock','month_year'])[['Neutral', 'Positive', 'Negative']].mean()

# Save the result to a new DataFrame
test_df = average_by_month_year.reset_index()

# Print the result DataFrame
# print(test_df)

# Concatenate vertically (add df2 to the bottom of df1)
SA_df = pd.concat([SA_df, test_df], axis=0)

# Reset the index
SA_df.reset_index(drop=True, inplace=True)

# Print the concatenated DataFrame
print(SA_df)

     Stock month_year   Neutral  Positive  Negative
0        A    2018-01  0.833321  0.166669  0.000010
1        A    2018-02  0.666658  0.333336  0.000007
2        A    2018-03  0.999707  0.000251  0.000041
3        A    2018-04  0.770916  0.193607  0.035477
4        A    2018-05  0.805719  0.115470  0.078811
...    ...        ...       ...       ...       ...
8006   ZTS    2019-08  0.452021  0.468811  0.079167
8007   ZTS    2019-09  0.416956  0.438070  0.144974
8008   ZTS    2019-10  0.537776  0.174727  0.287497
8009   ZTS    2019-11  0.545831  0.271496  0.182673
8010   ZTS    2019-12  0.344873  0.404221  0.250906

[8011 rows x 5 columns]


In [129]:
sa20_df = pd.read_csv(r'/Users/zoeyneo/code/zoeyneo/investoinsight/sentiment2020.csv')
sa20_df.head()

,Unnamed: 0,Headline,Date,Stock,Neutral,Positive,Negative
0,0,stocks that hit 52week highs on friday,2020-06-05,A,0.996414,1.911992e-05,0.003567
1,1,stocks that hit 52week highs on wednesday,2020-06-03,A,0.997737,2.388915e-05,0.002239
2,2,71 biggest movers from friday,2020-05-26,A,0.997286,1.869326e-04,0.002527
3,3,46 stocks moving in fridays midday session,2020-05-22,A,0.999996,9.135268e-08,0.000004
4,4,b of a securities maintains neutral on agilent...,2020-05-22,A,0.112909,8.801960e-01,0.006895


In [130]:
df = pd.DataFrame(sa20_df)

# Convert 'date' column 
df['date'] = pd.to_datetime(df['Date'], format='%Y-%m')

# Extract both month and year from the 'date' column
df['month'] = df['date'].dt.month
df['year'] = df['date'].dt.year

# Combine 'month' and 'year' into 'month_year'
df['month_year'] = df.apply(lambda x: f"{x['year']}-{x['month']:02d}", axis=1)

# Calculate the average score by month
average_by_month_year = df.groupby(['Stock','month_year'])[['Neutral', 'Positive', 'Negative']].mean()

# Save the result to a new DataFrame
test_df = average_by_month_year.reset_index()

# Print the result DataFrame
# print(test_df)

# Concatenate vertically (add df2 to the bottom of df1)
SA_df = pd.concat([SA_df, test_df], axis=0)

# Reset the index
SA_df.reset_index(drop=True, inplace=True)

# Print the concatenated DataFrame
print(SA_df)

      Stock month_year       Neutral      Positive  Negative
0         A    2018-01  8.333207e-01  1.666689e-01  0.000010
1         A    2018-02  6.666576e-01  3.333358e-01  0.000007
2         A    2018-03  9.997074e-01  2.514758e-04  0.000041
3         A    2018-04  7.709160e-01  1.936072e-01  0.035477
4         A    2018-05  8.057192e-01  1.154701e-01  0.078811
...     ...        ...           ...           ...       ...
10211   ZTS    2020-02  3.661496e-01  5.727494e-01  0.061101
10212   ZTS    2020-03  2.231895e-01  4.750676e-01  0.301743
10213   ZTS    2020-04  8.900490e-02  4.071743e-01  0.503821
10214   ZTS    2020-05  2.010897e-01  4.001261e-01  0.398784
10215   ZTS    2020-06  7.367237e-07  5.032118e-07  0.999999

[10216 rows x 5 columns]


In [198]:
sa21_df = pd.read_csv(r'/Users/zoeyneo/code/zoeyneo/investoinsight/sentiment2021.csv')
sa21_df

,Stock,Date,Neu,Pos,Neg
0,AAPL,2020-06-30,0.342230,6.064581e-01,0.051312
1,AAPL,2020-07-31,0.999988,4.835667e-07,0.000012
2,AAPL,2020-09-30,0.999900,2.876040e-06,0.000097
3,AAPL,2020-10-31,0.748453,1.041690e-01,0.147378
4,AAPL,2020-11-30,0.700595,9.277428e-02,0.206630
...,...,...,...,...,...
2309,ZS,2022-04-30,0.526264,2.793446e-01,0.194392
2310,ZS,2022-05-31,0.594872,3.249248e-01,0.080204
2311,ZS,2022-06-30,0.582619,3.393019e-01,0.078080
2312,ZS,2022-07-31,0.779860,1.282315e-01,0.091909


In [132]:
df = pd.DataFrame(sa21_df)

# Convert 'date' column 
df['date'] = pd.to_datetime(df['Date'], format='%Y-%m')
df.rename(columns={'Pos': 'Positive', 'Neu': 'Neutral', 'Neg': 'Negative'}, inplace=True)
df = df[['date','Stock', 'Neutral', 'Positive', 'Negative']]

# Extract both month and year from the 'date' column
df['month'] = df['date'].dt.month
df['year'] = df['date'].dt.year

# Combine 'month' and 'year' into 'month_year'
df['month_year'] = df.apply(lambda x: f"{x['year']}-{x['month']:02d}", axis=1)

# Calculate the average score by month
average_by_month_year = df.groupby(['Stock','month_year'])[['Neutral', 'Positive', 'Negative']].mean()

# Save the result to a new DataFrame
test_df = average_by_month_year.reset_index()

# Print the result DataFrame
# print(test_df)

# # Concatenate vertically (add df2 to the bottom of df1)
SA_df = pd.concat([SA_df, test_df], axis=0)

# Reset the index
SA_df.reset_index(drop=True, inplace=True)

# Print the concatenated DataFrame
print(SA_df)

      Stock month_year   Neutral  Positive  Negative
0         A    2018-01  0.833321  0.166669  0.000010
1         A    2018-02  0.666658  0.333336  0.000007
2         A    2018-03  0.999707  0.000251  0.000041
3         A    2018-04  0.770916  0.193607  0.035477
4         A    2018-05  0.805719  0.115470  0.078811
...     ...        ...       ...       ...       ...
12525    ZS    2022-04  0.526264  0.279345  0.194392
12526    ZS    2022-05  0.594872  0.324925  0.080204
12527    ZS    2022-06  0.582619  0.339302  0.078080
12528    ZS    2022-07  0.779860  0.128232  0.091909
12529    ZS    2022-08  0.660035  0.282772  0.057194

[12530 rows x 5 columns]


In [133]:
sa22_df = pd.read_csv(r'/Users/zoeyneo/code/zoeyneo/investoinsight/sentiment2022.csv')
sa22_df.head()

,Unnamed: 0,Date,Stock,pos_ma,neu_ma,neg_ma
0,0,2022-09-30,A,1.000000,5.829613e-08,8.049628e-09
1,1,2022-10-31,A,1.000000,5.497283e-08,7.521992e-09
2,2,2022-11-30,A,0.999996,4.380394e-06,5.380639e-08
3,3,2022-12-31,A,0.395974,6.039033e-01,1.226440e-04
4,4,2023-01-31,A,0.993029,1.702921e-03,5.268022e-03


In [134]:
df = pd.DataFrame(sa22_df)

# Convert 'date' column 
df['date'] = pd.to_datetime(df['Date'], format='%Y-%m')

df.rename(columns={'pos_ma': 'Positive', 'neu_ma': 'Neutral', 'neg_ma': 'Negative'}, inplace=True)
df = df[['date','Stock', 'Neutral', 'Positive', 'Negative']]

# Extract both month and year from the 'date' column
df['month'] = df['date'].dt.month
df['year'] = df['date'].dt.year

# Combine 'month' and 'year' into 'month_year'
df['month_year'] = df.apply(lambda x: f"{x['year']}-{x['month']:02d}", axis=1)

# Calculate the average score by month
average_by_month_year = df.groupby(['Stock','month_year'])[['Neutral', 'Positive', 'Negative']].mean()

# Save the result to a new DataFrame
test_df = average_by_month_year.reset_index()

# Print the result DataFrame
# print(test_df)

# Concatenate vertically (add df2 to the bottom of df1)
SA_df = pd.concat([SA_df, test_df], axis=0)

# Reset the index
SA_df.reset_index(drop=True, inplace=True)

# Print the concatenated DataFrame
print(SA_df)

      Stock month_year   Neutral  Positive  Negative
0         A    2018-01  0.833321  0.166669  0.000010
1         A    2018-02  0.666658  0.333336  0.000007
2         A    2018-03  0.999707  0.000251  0.000041
3         A    2018-04  0.770916  0.193607  0.035477
4         A    2018-05  0.805719  0.115470  0.078811
...     ...        ...       ...       ...       ...
18151   ZTS    2023-05  0.002581  0.997409  0.000010
18152   ZTS    2023-06  0.806474  0.135247  0.058278
18153   ZTS    2023-07  0.992489  0.000046  0.007465
18154   ZTS    2023-08  0.328452  0.338144  0.333404
18155   ZTS    2023-09  0.999981  0.000009  0.000010

[18156 rows x 5 columns]


In [135]:
SA_df.rename(columns={'Stock': 'symbol', 'month_year': 'date', 'Neutral': 'sa_neu', 'Positive': 'sa_pos', 'Negative': 'sa_neg'}, inplace=True)
print(SA_df)

      symbol     date    sa_neu    sa_pos    sa_neg
0          A  2018-01  0.833321  0.166669  0.000010
1          A  2018-02  0.666658  0.333336  0.000007
2          A  2018-03  0.999707  0.000251  0.000041
3          A  2018-04  0.770916  0.193607  0.035477
4          A  2018-05  0.805719  0.115470  0.078811
...      ...      ...       ...       ...       ...
18151    ZTS  2023-05  0.002581  0.997409  0.000010
18152    ZTS  2023-06  0.806474  0.135247  0.058278
18153    ZTS  2023-07  0.992489  0.000046  0.007465
18154    ZTS  2023-08  0.328452  0.338144  0.333404
18155    ZTS  2023-09  0.999981  0.000009  0.000010

[18156 rows x 5 columns]


In [192]:
SA_df

,symbol,date,sa_neu,sa_pos,sa_neg
0,A,2018-01,0.833321,0.166669,0.000010
1,A,2018-02,0.666658,0.333336,0.000007
2,A,2018-03,0.999707,0.000251,0.000041
3,A,2018-04,0.770916,0.193607,0.035477
4,A,2018-05,0.805719,0.115470,0.078811
...,...,...,...,...,...
18151,ZTS,2023-05,0.002581,0.997409,0.000010
18152,ZTS,2023-06,0.806474,0.135247,0.058278
18153,ZTS,2023-07,0.992489,0.000046,0.007465
18154,ZTS,2023-08,0.328452,0.338144,0.333404


In [193]:
# Merge the DataFrames horizontally based on 'date' and 'Symbol'
all_df = data_df.merge(SA_df, on=['date', 'symbol'], how='left')

# Fill missing values with NaN
all_df.fillna(np.nan, inplace=True)

# Print the merged DataFrame
all_df

       Unnamed: 0     date symbol  Dividend        volume  fed_funds_rate  \
0           29538  2018-07    HPQ      0.00  6.845513e+06            1.91   
1           29037  2018-08    CCL      0.50  3.182053e+06            1.91   
2           29042  2018-08    FDS      0.64  2.294333e+05            1.91   
3           29043  2018-08    FDX      0.00  1.211117e+06            1.91   
4           29045  2018-08    HPQ      0.00  6.493083e+06            1.91   
...           ...      ...    ...       ...           ...             ...   
13033       25460  2023-03    XYL      0.00  1.532100e+06            4.65   
13034       25461  2023-03    YUM      0.00  1.474093e+06            4.65   
13035       25462  2023-03    ZBH      0.24  1.147900e+06            4.65   
13036       25464  2023-03   ZION      0.00  5.970143e+06            4.65   
13037       25465  2023-03    ZTS      0.00  1.953233e+06            4.65   

               gdp  debt_to_equity       eps  return_on_equity  quick_ratio

In [197]:
all_df.isnull().sum()

Unnamed: 0               0
date                     0
symbol                   0
Dividend                 0
volume                   0
fed_funds_rate           0
gdp                      0
debt_to_equity           0
eps                      0
return_on_equity         0
quick_ratio              0
operating_ratio          0
inventory_turnover       0
stock_price              0
sa_neu                7467
sa_pos                7467
sa_neg                7467
dtype: int64

In [196]:
all_df.to_csv('data.csv', index=False)